# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,71.58,77,73,20.00,MU,1695606273
1,1,adamstown,-25.0660,-130.1015,68.90,76,100,27.49,PN,1695606262
2,2,jayapura,-2.5333,140.7000,87.19,76,94,4.90,ID,1695606407
3,3,uturoa,-16.7333,-151.4333,74.23,74,100,24.94,PF,1695606286
4,4,albany,42.6001,-73.9662,57.85,85,100,2.82,US,1695606065


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo = True, 
    c = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow down the DataFrame based on the criteria
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] > 69.8) & #21C = 69.8F
    (city_data_df['Max Temp'] < 80.6) & #27C = 80.6F
    (city_data_df['Wind Speed'] < 9) & #<4.5 is too restricted for my data
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_conditions_df.dropna(inplace=True)

# Display sample data
ideal_conditions_df

C:\Users\qianc\AppData\Local\Temp\ipykernel_29912\2761042094.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_conditions_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,figuig (centre),32.1089,-1.2286,77.16,25,0,3.27,MA,1695606414
183,183,laguna,38.4210,-121.4238,76.75,47,0,8.05,US,1695606425
184,184,at taj,24.2049,23.2857,73.87,26,0,6.67,LY,1695606425
280,280,siwa oasis,29.2041,25.5195,77.74,40,0,2.42,EG,1695606436
290,290,jalalabad,34.4265,70.4515,74.08,55,0,0.49,AF,1695606437
354,354,harnai,30.1005,67.9374,74.93,29,0,3.47,PK,1695606444
431,431,al kharijah,25.4514,30.5464,79.36,25,0,7.72,EG,1695606453
469,469,kill devil hills,36.0307,-75.6760,73.42,91,0,4.61,US,1695606325
483,483,paphos,34.7667,32.4167,73.18,33,0,6.91,CY,1695606459
500,500,siverek,37.7550,39.3167,75.00,23,0,2.71,TR,1695606461


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
81,figuig (centre),MA,32.1089,-1.2286,25,
183,laguna,US,38.4210,-121.4238,47,
184,at taj,LY,24.2049,23.2857,26,
280,siwa oasis,EG,29.2041,25.5195,40,
290,jalalabad,AF,34.4265,70.4515,55,
354,harnai,PK,30.1005,67.9374,29,
431,al kharijah,EG,25.4514,30.5464,25,
469,kill devil hills,US,36.0307,-75.6760,91,
483,paphos,CY,34.7667,32.4167,33,
500,siverek,TR,37.7550,39.3167,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "hotel",
    "limit": 1,  # Limit to one result (the nearest hotel)
    "sort": "distance",  # Sort by distance
    "apikey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance(point({longitude},{latitude}),{radius})"
    params["bias"] = f"point({longitude},{latitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
figuig (centre) - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
jalalabad - nearest hotel: No hotel found
harnai - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
paphos - nearest hotel: No hotel found
siverek - nearest hotel: No hotel found
rampura - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
81,figuig (centre),MA,32.1089,-1.2286,25,No hotel found
183,laguna,US,38.4210,-121.4238,47,No hotel found
184,at taj,LY,24.2049,23.2857,26,No hotel found
280,siwa oasis,EG,29.2041,25.5195,40,No hotel found
290,jalalabad,AF,34.4265,70.4515,55,No hotel found
354,harnai,PK,30.1005,67.9374,29,No hotel found
431,al kharijah,EG,25.4514,30.5464,25,No hotel found
469,kill devil hills,US,36.0307,-75.6760,91,No hotel found
483,paphos,CY,34.7667,32.4167,33,No hotel found
500,siverek,TR,37.7550,39.3167,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo = True, 
    c = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    tools=["hover"],
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"],
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)